In [ ]:
import pandas as pd
import psycopg2
from src.load_trafic_ import extract_data

# PostgreSQL database connection details
db_params = {
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'root',
    'database': 'traffic'
}


# Connect to PostgreSQL
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Reads CSV file into a DataFrame then 
# Load the data into a pandas DataFrame
constant_info_df ,observed_properties_df = extract_data()


# Create a table for constant vehicle information
constant_info_table_query = '''
CREATE TABLE IF NOT EXISTS constant_info_table (
    track_id SERIAL PRIMARY KEY,
    type VARCHAR,
    traveled_distance FLOAT,
    avg_speed FLOAT
);
'''
cur.execute(constant_info_table_query)
conn.commit()

# Insert data into constant_info_table
constant_info_df.to_sql('constant_info_table', conn, index_label='vehicle_id', if_exists='append')

# Create a table for observed properties with a foreign key to constant_info_table

observed_properties_table_query = '''
CREATE TABLE IF NOT EXISTS observed_properties_table (
    track_id SERIAL PRIMARY KEY,
    lat FLOAT,
    lon FLOAT,
    time TIMESTAMP,
    speed FLOAT,
    lon_acc FLOAT,
    lat_acc FLOAT
);
'''
cur.execute(observed_properties_table_query)
conn.commit()

# Insert data into observed_properties_table
observed_properties_df.to_sql('observed_properties_table', conn, index_label='track_id', if_exists='append')

# Commit and close
conn.commit()
conn.close()



